In [ ]:
import os
import numpy
import feets
import feets.preprocess
import pickle
import pandas
import django.as_table

os.chdir('../')
#Importing prompt and afterglow indexes using pickle
with open('StartandFinish.pickle','rb') as f:
    prompt_start, prompt_finish, afterglow_start, afterglow_finish = pickle.load(f)
os.chdir('Raw Data')
    
#Listing files and setting column names
files_list = os.listdir()
col_names = ['Time', 'Time_err_p', 'Time_err_n', 'Fluence', 'Fluence_err_p', 'Fluence_err_n']

Features_list = []

for i in range(2):
    

i=0
    #Creating the dataframe
    dataframe = pandas.read_csv(files_list[i], sep='\t', names=col_names, engine='python')
    dataframe = dataframe.drop(columns=['Time_err_p', 'Time_err_n'])
    dataset = dataframe.values
    #Extracting only the prompt and afterglow fluences in a single column
    prompt = dataset[prompt_start[i]:prompt_finish[i]]
    afterglow = dataset[afterglow_start[i]:afterglow_finish[i]]
    dataset = numpy.concatenate((prompt,afterglow))
    dataset = dataset.astype('float32')
    
    
    #Preprocess using feets
    features = []
    time, mag, error = feets.preprocess.remove_noise(dataset[:,0], dataset[:,1], dataset[:,2])
    lc = [time, mag, error]
    fs = feets.FeatureSpace(exclude = ['Color','Eta_color','Q31_color','StetsonJ', 'StetsonL'])
    features, values = fs.extract(*lc)
    as_table(features, values)
    features.append(features,values)


